# Sparkify: Customers Churn prediction

![Customers Churn](https://thinkml.ai/content/images/2019/11/image-10.png)

## Table of Contents

1. [Overview](#overview)
2. [Load and Clean Dataset](#load_clean)
    1. [Checking for missing values](#check)
    2. [Cleaning the data](#clean)
3. [Exploratory Data Analysis](./02_data_exploration.ipynb)
    1. [Descriptive statistics](#stat)
    2. [Defining churn indicator](#churn)
    2. [Data Exploration](#explore)
4. [Feature Engineering](./03_feature_engineering.ipynb)
5. [Model training and evaluation](./04_model_training_evaluation.ipynb)
6. [Conclusion](./05_conclusion.ipynb)

## 1. Overview<a id='overview'></a>
This notebooks is done in the context of the Capstone project for `Data Science Nanodegree` Program by Udacity.

Sparkify is a music streaming service like Spotify and Pandora. The users can use the service either the `Premium`  or the `Free Tier`. The premium plan with the monthly fees payment enables the use of service without any advertisements between songs.

The used data contains the user activity logs happening in the service. Those contain visited pages, service upgrade or downgrade events, events timestamps, demographic info, ...

In [1]:
! pip install -r requirements.txt

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, udf
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, DoubleType, LongType, IntegerType, DateType, TimestampType
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import pandas as pd

In [3]:
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

In [1]:
events_data_path = "../data/mini_sparkify_event_data.json"
%store events_data_path

Stored 'events_data_path' (str)


In [5]:
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
import utils

In [6]:
# Make the output more like pandas and less like command-line SQL
spark.sparkContext.getConf().getAll()

[('spark.driver.host', '192.168.1.176'),
 ('spark.app.id', 'local-1596899599143'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.name', 'Sparkify'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.port', '34375')]

In [7]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [8]:
spark

# <center> 2. Load and Clean Dataset <a id='load_clean'></a> </center>

1. Load the data.
2. Check for invalid or missing values e.g. records without userids or sessionids. 
3. Clean the data

In [9]:
# Load the Sparkify Events data from a JSON file into a Spark DataFrame
events_df = spark.read.json(events_data_path)
events_df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [10]:
nb_rows = events_df.count()
print("Number of rows: ", nb_rows)
print("Number of columns: ", len(events_df.columns))

Number of rows:  286500
Number of columns:  18


# Checking for missing data<a id='check'></a>

In [11]:
dd = events_df.select([count(when(isnan(c), c)).alias(c) for c in events_df.columns]).toPandas().T
dd[dd[0] > 0]

,0


The data set has no NaNs. In PySpark NaN is not the same as Null. Both of these are also different than an empty string "". So we check also Null values.

In [12]:
null_counts = events_df.select([
    count(when(col(column_name).isNull(), column_name)).alias(column_name)
        for column_name in events_df.columns
    ]).toPandas().T
null_counts

,0
artist,58392
auth,0
firstName,8346
gender,8346
itemInSession,0
lastName,8346
length,58392
level,0
location,8346
method,0


It appears that there are some users with misssing registration field, firstName and lastName. This might be related to non registered users.

### Checking rows with Null value in all columns

In [13]:
events_df.na.drop(how="all").count()

286500

It appears that there is no row with Null value in all columns 

### Check empty user IDs and sessionID

Regardless of the reason (logged out, unregistered...), empty userIds and sessionIds are useless for the purpose of detecting potentially churning users, so we can drop the corresponding records.

In [14]:
events_df.filter((
    events_df["userId"] == "") | events_df["userId"].isNull() | isnan(events_df["userId"])).count()

8346

In [15]:
events_df.filter((
    events_df["sessionId"] == "") | events_df["sessionId"].isNull() | isnan(events_df["sessionId"])).count()
events_df.dropna(how = 'any', subset = ['userId', 'sessionId'])
events_df.count()

286500

Checking for empty sessionId values shows that there are no missing values for sessionId column

### Number of distinct users

In [16]:
events_df = events_df.filter(events_df.userId != '')
events_df.select('userId').distinct().count()

225

### Check for Duplicated events logs
Comparing events_df.count() to events_df.distinct().count() shows that there are no duplicates

In [17]:
events_df.distinct().count()

278154

### Let's re-check some of the empty fields after the cleaning
e.g. registration, gender, ...

In [18]:
null_counts = events_df.select([count(when(col(column_name).isNull(), column_name)).alias(column_name) for column_name in 
           events_df.columns]).toPandas().T
null_counts.sort_values(by=0, axis=0, ascending=False)

,0
artist,50046
song,50046
length,50046
page,0
userAgent,0
ts,0
status,0
sessionId,0
registration,0
method,0


<div class="alert alert-block alert-warning">
It appears that the rows with missing values where related to the log events with no userId
</div>

In [19]:
pd.DataFrame(events_df.take(3), columns=events_df.columns).head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Martha Tilston,Logged In,Colin,M,50,Freeman,277.89016,paid,"Bakersfield, CA",PUT,NextSong,1538173362000,29,Rockpools,200,1538352117000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
1,Five Iron Frenzy,Logged In,Micah,M,79,Long,236.09424,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1538331630000,8,Canada,200,1538352180000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
2,Adam Lambert,Logged In,Colin,M,51,Freeman,282.82730,paid,"Bakersfield, CA",PUT,NextSong,1538173362000,29,Time For Miracles,200,1538352394000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30


In [20]:
events_df.count()

278154